In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import time
import pandas as pd
import pickle

In [110]:
ratings = pd.read_csv( 'ml-latest/ratings.csv', sep = ',', engine = 'python' )

In [111]:
ratings.describe()

,userId,movieId,rating,timestamp
count,2.602429e+07,2.602429e+07,2.602429e+07,2.602429e+07
mean,1.350371e+05,1.584911e+04,3.528090e+00,1.171258e+09
std,7.817620e+04,3.108526e+04,1.065443e+00,2.052889e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,6.716400e+04,1.073000e+03,3.000000e+00,9.907545e+08
50%,1.351630e+05,2.583000e+03,3.500000e+00,1.151716e+09
75%,2.026930e+05,6.503000e+03,4.000000e+00,1.357578e+09
max,2.708960e+05,1.762750e+05,5.000000e+00,1.501830e+09


In [30]:
movies = pd.read_csv( 'ml-latest/movies.csv', sep = ',', engine = 'python' )

In [27]:
f = open('movie_data.pkl', 'rb')
movies_enriched = pickle.load(f)
f.close()
movies_not_enriched = {} 

In [28]:
#len(movies_enriched)
max_id = max(movies_enriched, key=int)
print(max_id)

34231


In [31]:
nrows = movies.shape[0]
i = 1
for index, row in movies.iterrows():
    if row['movieId'] < max_id:
        print(row['movieId'])
        i = i + 1
        continue
    if i % 5 == 0:
        print(str(i) + '/' + str(nrows))
    if i % 100 == 0:
        f = open('movie_data.pkl', 'wb')
        print('saving pickle')
        pickle.dump(movies_enriched, f)
        f.close()
    title = row['title']
    real_title = title[:-7]
    year = title[-5:-1]

    if real_title[-5:].lower() == ', the':
        real_title = 'the ' + real_title[:-5]
    movie_data = get_movie_data(real_title, year)
    if not movie_data:
        movies_not_enriched[row['movieId']] = row['title']
    else:
        movies_enriched[row['movieId']] = {
            'data': movie_data,
            'genres': row['genres']
        }
    i = i + 1


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279

2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313


4247
4248
4249
4250
4251
4252
4253
4254
4255
4256
4257
4258
4259
4260
4261
4262
4263
4265
4266
4267
4268
4269
4270
4271
4272
4273
4274
4275
4276
4277
4278
4279
4280
4281
4282
4283
4284
4285
4286
4287
4288
4289
4290
4291
4292
4293
4294
4295
4296
4297
4298
4299
4300
4301
4302
4303
4304
4305
4306
4307
4308
4309
4310
4311
4312
4313
4314
4315
4316
4317
4318
4319
4320
4321
4322
4323
4324
4325
4326
4327
4328
4329
4330
4331
4332
4333
4334
4335
4336
4337
4338
4339
4340
4341
4342
4343
4344
4345
4346
4347
4348
4349
4350
4351
4352
4353
4354
4355
4356
4357
4358
4359
4360
4361
4362
4363
4364
4365
4366
4367
4368
4369
4370
4371
4372
4373
4374
4375
4376
4377
4378
4379
4380
4381
4382
4383
4384
4385
4386
4387
4388
4389
4390
4391
4392
4393
4394
4395
4396
4397
4398
4399
4400
4401
4402
4403
4404
4405
4406
4407
4408
4409
4410
4411
4412
4413
4414
4415
4416
4417
4418
4419
4420
4421
4422
4423
4424
4425
4426
4427
4428
4429
4430
4431
4432
4433
4434
4435
4436
4437
4438
4439
4440
4441
4442
4443
4444
4445
4446
4447


6680
6681
6682
6683
6684
6685
6686
6687
6688
6689
6690
6691
6692
6693
6694
6695
6696
6697
6698
6699
6700
6701
6702
6703
6704
6705
6706
6707
6708
6709
6710
6711
6712
6713
6714
6715
6716
6717
6718
6719
6720
6721
6722
6723
6724
6725
6726
6727
6728
6729
6730
6731
6732
6733
6734
6735
6736
6737
6738
6739
6740
6741
6742
6743
6744
6745
6746
6747
6748
6749
6750
6751
6752
6753
6754
6755
6756
6757
6758
6759
6760
6761
6762
6763
6764
6765
6766
6767
6768
6769
6770
6771
6772
6773
6774
6775
6776
6777
6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879


26157
26158
26159
26160
26162
26163
26164
26167
26169
26170
26171
26172
26174
26175
26176
26177
26178
26180
26181
26183
26184
26185
26187
26188
26189
26191
26194
26195
26197
26198
26199
26202
26203
26204
26205
26206
26207
26208
26209
26210
26211
26213
26214
26215
26216
26219
26220
26221
26222
26223
26225
26226
26227
26228
26229
26230
26231
26232
26235
26236
26237
26240
26241
26242
26243
26245
26246
26247
26248
26249
26251
26252
26253
26254
26255
26256
26257
26258
26259
26263
26264
26265
26266
26267
26268
26269
26270
26271
26274
26277
26280
26282
26283
26285
26287
26288
26289
26290
26291
26294
26295
26297
26299
26301
26302
26303
26304
26306
26307
26308
26309
26312
26313
26314
26315
26316
26317
26318
26320
26321
26322
26323
26324
26325
26326
26327
26333
26334
26336
26337
26338
26339
26340
26341
26342
26343
26344
26345
26346
26347
26349
26350
26353
26354
26356
26357
26359
26360
26362
26364
26365
26366
26367
26368
26369
26370
26371
26372
26375
26377
26378
26379
26380
26383
26386
26387
2638

...

32595/45843
32600/45843
saving pickle
32605/45843
32610/45843
32615/45843
32620/45843
32625/45843
32630/45843
32635/45843
32640/45843
32645/45843
32650/45843
32655/45843
32660/45843
32665/45843
32670/45843
32675/45843
32680/45843
32685/45843
32690/45843
32695/45843
32700/45843
saving pickle
32705/45843
32710/45843
32715/45843
32720/45843
32725/45843
32730/45843
32735/45843
32740/45843
32745/45843
32750/45843
32755/45843
32760/45843
32765/45843
32770/45843
32775/45843
32780/45843
32785/45843
32790/45843
32795/45843
32800/45843
saving pickle
32805/45843
32810/45843
32815/45843
32820/45843
32825/45843
32830/45843
32835/45843
32840/45843
32845/45843
32850/45843
32855/45843
32860/45843
32865/45843
32870/45843
32875/45843
32880/45843
32885/45843
32890/45843
32895/45843
32900/45843
saving pickle
32905/45843
32910/45843
32915/45843
32920/45843
32925/45843
32930/45843
32935/45843
32940/45843
32945/45843
32950/45843
32955/45843
32960/45843
32965/45843
32970/45843
32975/45843
32980/45843
32985/45

35820/45843
35825/45843
35830/45843
35835/45843
35840/45843
35845/45843
35850/45843
35855/45843
35860/45843
35865/45843
35870/45843
35875/45843
35880/45843
35885/45843
35890/45843
35895/45843
35900/45843
saving pickle
35905/45843
35910/45843
35915/45843
35920/45843
35925/45843
35930/45843
35935/45843
35940/45843
35945/45843
35950/45843
35955/45843
35960/45843
35965/45843
35970/45843
35975/45843
35980/45843
35985/45843
35990/45843
35995/45843
36000/45843
saving pickle
36005/45843
36010/45843
36015/45843
36020/45843
36025/45843
36030/45843
36035/45843
36040/45843
36045/45843
36050/45843
36055/45843
36060/45843
36065/45843
36070/45843
36075/45843
36080/45843
36085/45843
36090/45843
36095/45843
36100/45843
saving pickle
36105/45843
36110/45843
36115/45843
36120/45843
36125/45843
36130/45843
36135/45843
36140/45843
36145/45843
36150/45843
36155/45843
36160/45843
36165/45843
36170/45843
36175/45843
36180/45843
36185/45843
36190/45843
36195/45843
36200/45843
saving pickle
36205/45843
36210/45

39050/45843
39055/45843
39060/45843
39065/45843
39070/45843
39075/45843
39080/45843
39085/45843
39090/45843
39095/45843
39100/45843
saving pickle
39105/45843
39110/45843
39115/45843
39120/45843
39125/45843
39130/45843
39135/45843
39140/45843
39145/45843
39150/45843
39155/45843
39160/45843
39165/45843
39170/45843
39175/45843
39180/45843
39185/45843
39190/45843
39195/45843
39200/45843
saving pickle
39205/45843
39210/45843
39215/45843
39220/45843
39225/45843
39230/45843
39235/45843
39240/45843
39245/45843
39250/45843
39255/45843
39260/45843
39265/45843
39270/45843
39275/45843
39280/45843
39285/45843
39290/45843
39295/45843
39300/45843
saving pickle
39305/45843
39310/45843
39315/45843
39320/45843
39325/45843
39330/45843
39335/45843
39340/45843
39345/45843
39350/45843
39355/45843
39360/45843
39365/45843
39370/45843
39375/45843
39380/45843
39385/45843
39390/45843
39395/45843
39400/45843
saving pickle
39405/45843
39410/45843
39415/45843
39420/45843
39425/45843
39430/45843
39435/45843
39440/45

42280/45843
42285/45843
42290/45843
42295/45843
42300/45843
saving pickle
42305/45843
42310/45843
42315/45843
42320/45843
42325/45843
42330/45843
42335/45843
42340/45843
42345/45843
42350/45843
42355/45843
42360/45843
42365/45843
42370/45843
42375/45843
42380/45843
42385/45843
42390/45843
42395/45843
42400/45843
saving pickle
42405/45843
42410/45843
42415/45843
42420/45843
42425/45843
42430/45843
42435/45843
42440/45843
42445/45843
42450/45843
42455/45843
42460/45843
42465/45843
42470/45843
42475/45843
42480/45843
42485/45843
42490/45843
42495/45843
42500/45843
saving pickle
42505/45843
42510/45843
42515/45843
42520/45843
42525/45843
42530/45843
42535/45843
42540/45843
42545/45843
42550/45843
42555/45843
42560/45843
42565/45843
42570/45843
42575/45843
42580/45843
42585/45843
42590/45843
42595/45843
42600/45843
saving pickle
42605/45843
42610/45843
42615/45843
42620/45843
42625/45843
42630/45843
42635/45843
42640/45843
42645/45843
42650/45843
42655/45843
42660/45843
42665/45843
42670/45

45505/45843
45510/45843
45515/45843
45520/45843
45525/45843
45530/45843
45535/45843
45540/45843
45545/45843
45550/45843
45555/45843
45560/45843
45565/45843
45570/45843
45575/45843
45580/45843
45585/45843
45590/45843
45595/45843
45600/45843
saving pickle
45605/45843
45610/45843
45615/45843
45620/45843
45625/45843
45630/45843
45635/45843
45640/45843
45645/45843
45650/45843
45655/45843
45660/45843
45665/45843
45670/45843
45675/45843
45680/45843
45685/45843
45690/45843
45695/45843
45700/45843
saving pickle
45705/45843
45710/45843
45715/45843
45720/45843
45725/45843
45730/45843
45735/45843
45740/45843
45745/45843
45750/45843
45755/45843
45760/45843
45765/45843
45770/45843
45775/45843
45780/45843
45785/45843
45790/45843
45795/45843
45800/45843
saving pickle
45805/45843
45810/45843
45815/45843
45820/45843
45825/45843
45830/45843
45835/45843
45840/45843


In [36]:

f = open('movie_data.pkl', 'wb')
print('saving pickle')
pickle.dump(movies_enriched, f)
f.close()

saving pickle


In [10]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
def get_movie_data(name, year):
    query = '''
    SELECT ?label ?film ?Rdate ?imdb ?abstract ?director ?country ?writer (group_concat(?star;separator='||') as ?cast) WHERE { 
        ?film rdf:type dbo:Film . 
        ?film rdfs:label ?label . 
        ?film dbo:abstract ?abstract .
        OPTIONAL { ?film dbo:releaseDate ?Rdate } .
        OPTIONAL { ?film dbo:imdbId ?imdb } . 
        OPTIONAL { ?film dbo:director ?director } . 
        OPTIONAL { ?film dbp:country ?country } . 
        OPTIONAL { ?film dbo:writer ?writer } . 
        OPTIONAL { ?film dbo:starring ?star } . 
        FILTER (lang(?label) = \'en\') .
        FILTER (lang(?abstract) = "en")
        filter contains(lcase(?label), "''' + str(name).lower() + '''") .
        filter contains(?abstract, "''' + str(year) + '''")} group by ?label ?film ?Rdate ?imdb ?abstract ?director ?country ?writer LIMIT 50
    '''
    sparql.setQuery(query)  # the previous query as a literal string
    return sparql.query().convert()['results']['bindings']

get_movie_data('Jumanji', 1995)

[{'abstract': {'type': 'literal',
   'value': "Jumanji is a 1995 American family adventure film directed by Joe Johnston. It is an adaptation of the 1981 children's book of the same name by Chris Van Allsburg. The film was written by Greg Taylor, Jonathan Hensleigh, and Jim Strain, and stars Robin Williams, Kirsten Dunst, Bradley Pierce, David Alan Grier, Bonnie Hunt, Jonathan Hyde, and Bebe Neuwirth. The special effects were provided by Industrial Light & Magic for computer graphic elements and Amalgamated Dynamics for animatronics components. The film was dedicated to visual effects supervisor Stephen L. Price, who died before the film's release. The story centers on young Alan Parrish, who becomes trapped in a board game while playing with his best friend Sarah Whittle in 1969. Twenty-six years later in 1995, siblings Judy and Peter Shepherd find the game, begin playing and then unwittingly release the now-adult Alan. After tracking down Sarah, the quartet resolve to finish the game

In [107]:

start = time.time()
print(get_movie_data('some like it hot', 1959))
end = time.time()
print(end - start)

[{'label': {'type': 'literal', 'xml:lang': 'en', 'value': 'Some Like It Hot'}, 'film': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Some_Like_It_Hot'}, 'abstract': {'type': 'literal', 'xml:lang': 'en', 'value': "Some Like It Hot is a 1959 American romantic comedy film set in 1929, directed and produced by Billy Wilder, starring Marilyn Monroe, Tony Curtis, and Jack Lemmon. The supporting cast includes George Raft, Pat O'Brien, Joe E. Brown, Joan Shawlee, and Nehemiah Persoff. The plot is based on a screenplay by Billy Wilder and Michael Logan from the French film Fanfare of Love. The film is about two musicians who dress in drag in order to escape from mafia gangsters whom they witnessed commit the Saint Valentine's Day Massacre. The film was produced in black and white, even though color films were increasing in popularity. Some Like It Hot is considered to be one of the greatest film comedies of all time. It was voted as the top comedy film by the American Film Institute on 